In [8]:
import random
import h5py
import numpy as np
import tqdm
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

In [9]:
sample_size = 512
file = h5py.File('output_digi_HDF_Mg22_Ne20pp_8MeV.h5', 'r')
keys = list(file.keys())
length = len(keys)

In [10]:
#making an array of the lengths of events
event_lens = np.zeros(length, int)
for i in range(length):
    event = keys[i]
    event_lens[i] = len(file[event])

In [ ]:
#making an array of the events data-- [event #, instance, data value]
event_data = np.zeros((length, np.max(event_lens), 12), float)
for n in tqdm.tqdm(range(len(keys))):
    name = keys[n]
    event = file[name]
    ev_len = len(event)
    #converting event into an array
    for i,e in enumerate(event):
        instant = np.array(list(e))
        event_data[n][i][:] = np.array(instant)    
np.save('Mg22_Ne20pp', event_data)

In [18]:
#NOT completely random sampling!
data = np.load('Mg22_Ne20pp.npy')
file_name = 'Mg22_size' + str(sample_size) + '.h5'
new = h5py.File(file_name, 'w')
for i in tqdm.tqdm(range(length)):
    ev_len = event_lens[i]    #length of event-- i.e. number of instances
    particle_ids = data[i,:event_lens[i],5]
    label, distr = np.unique(particle_ids, return_counts=True)
    shortest = label[np.argmin(distr)]
    shortest_ind = np.argwhere(particle_ids == shortest)
    new_event = np.zeros((sample_size, 13), float)    #empty array for sampled event data
    if ev_len == sample_size:    #if array is already preferred length
        new_event[:,:-1] = data[i,:ev_len,:]
    else:
        for n in range(sample_size):    #the first instances sampled will be those belonging to the shortest track
            if n < shortest.size:
                new_event[n,:-1] = data[i,shortest_ind[n],:]
            else:
                row = random.randint(0, ev_len - 1)
                new_event[n,:-1] = data[i,row,:] 
    new_event[:,:2] /= 29.2     #scaling x and y coords to 1, rather than 29.2 cm
    new_event[:,2] /= 1000    #scaling z coord to 1, rather than 1000 cm 
    unique_point_ids = np.unique(data[i,:ev_len,5])    #array of unique particle IDs
    new_event[0,-1] = unique_point_ids.size - 1    #number of unique particles, scaled to start at 0
    new.create_dataset(keys[i], data = new_event)    #creating new dataset within the h5 file for the event

100%|██████████| 10000/10000 [00:16<00:00, 607.07it/s]


In [19]:
#cheking how the distribution of labels changes from sampling
name = 'Mg22_size' + str(sample_size) + '.h5'
data = h5py.File(name, 'r')
keys = list(data.keys())
real_tracks = np.zeros(len(keys),int)
sampled_tracks = np.zeros(len(keys),int)
for i in range(len(keys)):
    event = data[keys[i]]
    real_tracks[i] = event[0,-1]
    unique_point_ids = np.unique(event[:,5])    #array of unqiue particles IDs
    sampled_tracks[i] = unique_point_ids.size - 1
label, og_distr = np.unique(real_tracks, return_counts=True)
label, new_distr = np.unique(sampled_tracks, return_counts=True)
print(og_distr)
print(new_distr)
print(np.sum(np.abs(new_distr - og_distr))//2)

[5002  104 2554 2319   16    5]
[5002  104 2556 2317   16    5]
2


In [8]:
#creating data sets with fewer beam events and no 2, 5, or 6 track events
name = 'Mg22_size' + str(sample_size)
data = h5py.File(name + '.h5', 'r')
keys = list(data.keys())
length = len(keys)
new = h5py.File(name + '_edited.h5', 'w')
for i in tqdm.tqdm(range(length)):
    event = data[keys[i]]
    unique_point_ids = np.unique(event[:,5])    
    current_tracks = unique_point_ids.size - 1 
    og_tracks = event[0,-1]
    bad_events = np.array([1,4,5])
    #omitting incorrectly labeled events, the latter half of beam events, and 2, 5, and 6 track events
    if current_tracks != og_tracks or (og_tracks == 0 and i > 5000) or np.any(og_tracks == bad_events[:]): 
        continue
    else:
        new.create_dataset(keys[i], data = event)

100%|██████████| 10000/10000 [00:15<00:00, 625.31it/s]


In [20]:
#creating data sets with only 4-track events
name = 'Mg22_size' + str(sample_size)
data = h5py.File(name + '.h5', 'r')
keys = list(data.keys())
length = len(keys)
new = h5py.File(name + '_4-track.h5', 'w')
for i in tqdm.tqdm(range(length)):
    event = data[keys[i]]
    new_event = event[:,:]
    new_event[:,5] -= 1   #scaling IDs so they start at zero
    unique_point_ids = np.unique(event[:,5])    
    current_tracks = unique_point_ids.size - 1 
    og_tracks = event[0,-1]
    #omitting incorrectly labeled events, the latter half of beam events, and 2, 5, and 6 track events
    if og_tracks == 3 and og_tracks == current_tracks: 
        new.create_dataset(keys[i], data = new_event)
    else:
        continue

100%|██████████| 10000/10000 [00:07<00:00, 1360.48it/s]


In [22]:
#cheking the distribution
data = h5py.File('Mg22_size' + str(sample_size) + '_4-track.h5', 'r')
keys = list(data.keys())
num_tracks = np.zeros(len(keys),int)
for i in range(len(keys)):
    event = data[keys[i]]
    num_tracks[i] = int(event[0,12])
print(np.unique(num_tracks, return_counts=True))

(array([3]), array([2317]))


In [11]:
#setting aside a test set from the all events dataset
name = 'Mg22_size' + str(sample_size) # + '_4-track'
whole = h5py.File(name + '.h5','r')
test = h5py.File(name + '_test.h5', 'w')
rest = h5py.File(name + '_rest.h5', 'w')
keys = list(whole.keys())
length = len(keys)
test_len = int(0.2*length)
test_set_indices = np.random.choice(range(length), test_len, replace=False)
for i in tqdm.tqdm(range(length)):
    event = whole[keys[i]]
    if np.isin(i, test_set_indices, assume_unique=True):
        test.create_dataset(keys[i], data = event)
    else:
        rest.create_dataset(keys[i], data = event)

100%|██████████| 10000/10000 [00:08<00:00, 1134.85it/s]


In [12]:
#splits remaining events data into training and validation sets
name = 'Mg22_size' + str(sample_size) # + '_4-track'
whole = h5py.File(name + '_rest.h5','r')
train = h5py.File(name + '_train.h5', 'w')
val = h5py.File(name + '_val.h5', 'w')
keys = list(whole.keys())
length = len(keys)
val_len = int(0.25*length)     #20% of 80%
val_set_indices = np.random.choice(range(length), val_len, replace=False)
for i in tqdm.tqdm(range(length)):
    event = whole[keys[i]]
    if np.isin(i, val_set_indices, assume_unique=True):
        val.create_dataset(keys[i], data = event)
    else:
        train.create_dataset(keys[i], data = event)

100%|██████████| 8000/8000 [00:05<00:00, 1377.29it/s]


In [14]:
data = h5py.File('Mg22_size' + str(sample_size) + '_4-track_test.h5', 'r')
keys = list(data.keys())
length = len(keys)
point_ids = np.zeros((length,4),int)
counts = np.zeros((length,4),int)
for i in range(len(keys)):
    event = data[keys[i]]
    point_ids[i], counts[i] = np.unique(event[:,5], return_counts=True)
point_ids = point_ids.flatten()
counts = counts.flatten()
zeros = np.sum(counts[range(0,counts.size, 4)])
ones = np.sum(counts[range(1,counts.size, 4)])
twos = np.sum(counts[range(2,counts.size, 4)])
threes = np.sum(counts[range(3,counts.size, 4)])
print(np.unique(point_ids, return_counts=True))
print(zeros,ones,twos,threes)
print(zeros+ones+twos+threes)

(array([0, 1, 2, 3]), array([463, 463, 463, 463]))
70172 8646 57479 100759
237056
